# 1.この課題の目的
スクラッチを通してRNNの基礎を理解する
以下の要件をすべて満たしていた場合、合格とします。

※Jupyter Notebookを使い課題に沿った検証や説明ができている。

# 2.スクラッチによる実装
NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

Sprint11で作成したディープニューラルネットワークのクラスを拡張する形でRNNを作成します。

# 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

## 使用クラスのインポート

In [1]:
import numpy as np
import pandas as pd

## SimpleRNN

In [2]:
class SimpleRNN:
    """
    ノード数n_nodes1からn_nodes2へのRNN
    Parameters
    ----------
    n_features : int
        特徴量の数
    n_nodes : int
        ノード数
    activate : インスタンス
        最適化を行うクラスのインスタンス
    initializer : インスタンス
        初期化方法のインスタンス
        
    Atribute
    ----------
    self.Wx : numpy配列
        入力に対する重み
    self.Wh : numpy配列
        前の時刻から伝わる順伝播に対する重み
    self.B : numpy配列
        バイアス
    """
    def __init__(self, n_features, n_nodes, activate, initializer):
        self.activate = activate
        self.initializer = initializer
        self.Wx = self.initializer.W(n_features, n_nodes)
        self.Wh = self.initializer.W(n_nodes, n_nodes)
        self.B = self.initializer.B()

        
    def forward(self, Xt, forward_Ht=None):
        """
        フォワード
        Parameters
        ----------
        Xt : numpy配列
            入力データ
        forward_Ht : numpy配列
            時刻t-1の状態（前の時刻から伝わる順伝播）
            
        Returns
        ----------
        Ht : numpy配列
            出力
        """ 
        
        # 前の時刻から伝わる順伝播の有無を確認
        if forward_Ht is not None:
            # あり
            At = Xt @ self.Wx + forward_Ht @ self.Wh + self.B
        else:
            # なし
            At = Xt @ self.Wx + self.B
        
        # 活性化関数クラスのインスタンスで順伝搬
        Ht = self.activate.forward(At)
        
        return Ht

## Tanhクラス

In [3]:
class Tanh():
    
    def __init__(self):
        pass
    
    def forward(self, A):
        """
        Tanhの計算を実施
        Parameters
        ----------
        A : numpy配列
            入力データ
            
        Returns
        ----------
        tanh_A: numpy配列
            出力
        """ 
        
        # 計算結果を格納しreturnする
        tanh_A = np.tanh(A)

        return tanh_A

## SimpleInitializerクラス

In [4]:
class SimpleInitializer():
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    self.sigma : float
      ガウス分布の標準偏差
    """
    
    def __init__(self, sigma):

        self.sigma = sigma

        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          作成する配列の行数
        n_nodes2 : int
          作成する配列の列数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)

        return W
    

    def B(self):
        """
        バイアスの初期化

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(1)

        return B

# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

```python
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])
```
フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。
```python
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])
```

### 問題2では重みとバイアスが指定されている為、以下の専用のクラスを使用しフォワードプロパゲーションの動作確認を行う

## Test_SimpleRNNクラス

In [5]:
class Test_SimpleRNN:
    """
    ノード数n_nodes1からn_nodes2へのRNN
    Parameters
    ----------
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数

    batch_size : バッチサイズ7
    n_features : 入力特徴量
    n_nodes : RNNのノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, Wx, Wh, B, activate, initializer=None):
        self.activate = activate
        self.initializer = initializer
        self.Wx = Wx
        self.Wh = Wh
        self.B = B
        
        
    def forward(self, Xt, forward_Ht=None):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """ 
        
        if forward_Ht is not None:
            At = Xt @ self.Wx + forward_Ht @ self.Wh + self.B
        else:
            At = Xt @ self.Wx + self.B
        
        Ht = self.activate.forward(At)
        
        return Ht

## 初期値設定

In [6]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

## forwardの動作確認

### for文を使用した場合

In [7]:
RNN = Test_SimpleRNN(Wx=w_x, Wh=w_h, B=b, activate=Tanh())

In [21]:
for sequence in range(x.shape[1]):
    
    # 時刻0の時は前の時刻から伝わる順伝播がない
    if sequence == 0:
        # 時刻0→前の時刻から伝わる順伝播がない場合の計算
        RNN_out = RNN.forward(x[0][sequence], None)
    else:
        # 時刻0以外→前の時刻から伝わる順伝播がある場合の計算
        RNN_out = RNN.forward(x[0][sequence], RNN_out)
        
print('RNN_out = {}'.format(RNN_out))

RNN_out = [0.79494228 0.81839002 0.83939649 0.85584174]


### インスタンスを複数作成した場合

In [9]:
RNN1 = Test_SimpleRNN(Wx=w_x, Wh=w_h, B=b, activate=Tanh())
RNN2 = Test_SimpleRNN(Wx=w_x, Wh=w_h, B=b, activate=Tanh())
RNN3 = Test_SimpleRNN(Wx=w_x, Wh=w_h, B=b, activate=Tanh())

In [10]:
RNN1_out = RNN1.forward(x[0][0], None)
RNN2_out = RNN2.forward(x[0][1], RNN1_out)
RNN3_out = RNN3.forward(x[0][2], RNN2_out)
print('RNN3_out = {}'.format(RNN3_out))

RNN3_out = [0.79494228 0.81839002 0.83939649 0.85584174]
